# Notebook to Predict Delays in Flights for SCL Airport

In [ ]:
import warnings
from datetime import datetime

import holidays
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")


# import numpy as np

sns.set_theme(style="darkgrid")
# import sklearn

## 0. Load dataset
Create DataFrame, check data and format

Column details for file `dataset_SCL.csv`:

- **Fecha-I** : Date and time of the programmed flight.
- **Vlo-I** : Programmed flight number.
- **Ori-I** : City code of programmed origin.
- **Des-I** : City code of programmed destination.
- **Emp-I** : Airline code of programmed flight.
- **Fecha-O** : Date and time of operated flight.
- **Vlo-O** : Operated flight number.
- **Ori-O** : City code of operated origin.
- **Des-O** : City code of operated destination.
- **Emp-O** : Airline code of operated flight.
- **DIA** : Day number of operated flight.
- **MES** : Month number of operated flight.
- **AÑO** : Year of operated flight.
- **DIANOM** : Day of the week of operated flight.
- **TIPOVUELO** : Type of flight, I =International, N =National.
- **OPERA** : Name of the operated airline.
- **SIGLAORI** : Origin city name.
- **SIGLADES** : Destination city name.

In [ ]:
# Load dataset as Panda's DataFrame

dataset = pd.read_csv("input/dataset_SCL.csv")
dataset.head()

In [ ]:
# Show data shape as: (rows, columns)

dataset.shape

In [ ]:
# Check for duplicated rows

print(f"Are there duplicates?: {len(dataset.drop_duplicates()) != len(dataset)}")

In [ ]:
# search for missing values (NaN's)

dataset.isna().sum()

There's only 1 missing value from one column, corresponding to an empy value for the column `Vlo-I : Operated flight number`. 

To avoid removing the whole row of data, we will analyze the frequency in which `Vl-O` is equal to `Vlo-I`.

In [ ]:
# How often is 'Vlo-I' equal to 'Vlo-O' ?

fraction = (dataset["Vlo-I"] == dataset["Vlo-O"]).sum() / len(dataset)

print(f"'Vlo-I' is equal to 'Vlo-O' {fraction:.1%} of the time")

There is a very high probability (over 97%) that 'Vlo-O' can be correctly estimated from 'Vlo-I', so we decide to fill this value instead of removing the row.

In [ ]:
# fill missinf values for 'Vlo-O' from column 'Vlo-I'
dataset["Vlo-O"].fillna(dataset["Vlo-I"], inplace=True)
# count NaN's
dataset.isna().sum().sum()

In [ ]:
# Check min-max values for Day, Month and Year

dataset.describe()

In [ ]:
# Show unique values for each column to better undestand the data

print("Unique values for column 'Ori-I':", set(dataset["Ori-I"]), "\n")
print("Unique values for column 'Des-I':", set(dataset["Des-I"]), "\n")
print("Unique values for column 'Emp-I':", set(dataset["Emp-I"]), "\n")
print("Unique values for column 'Ori-O':", set(dataset["Ori-O"]), "\n")
print("Unique values for column 'Des-O':", set(dataset["Des-O"]), "\n")
print("Unique values for column 'Emp-O':", set(dataset["Emp-O"]), "\n")
print("Unique values for column 'DIANOM':", set(dataset["DIANOM"]), "\n")
print("Unique values for column 'TIPOVUELO':", set(dataset["TIPOVUELO"]), "\n")
print("Unique values for column 'OPERA':", set(dataset["OPERA"]), "\n")
print("Unique values for column 'SIGLAORI':", set(dataset["SIGLAORI"]), "\n")
print("Unique values for column 'SIGLADES':", set(dataset["SIGLADES"]), "\n")

In [ ]:
# transform dates from string into datetime object to calculate time difference
dataset["Fecha-I"] = dataset["Fecha-I"].apply(
    lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
)
dataset["Fecha-O"] = dataset["Fecha-O"].apply(
    lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
)

In [ ]:
dataset.head()

## 1. Analyze Distribution of Data

In [ ]:
# Distribution of Flights vs Day of the Week

g = sns.catplot(
    x="DIANOM",
    data=dataset,
    kind="count",
    order=["Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado", "Domingo"],
)
g.set_axis_labels("\nDay of the Week\n", "\nTotal Flights\n", size=16)
g.fig.suptitle("\nNumber of Flights per Day of the Week", size=20, y=1.15)
g.figure.set_size_inches(15, 5)

In [ ]:
# Distribution of Flights vs Month

g = sns.catplot(x="MES", data=dataset, kind="count", order=range(1, 13))
g.set_axis_labels("\nMonth\n", "\nTotal Flights\n", size=16)
g.fig.suptitle("\nNumber of Flights per Month", size=20, y=1.15)
g.figure.set_size_inches(15, 5)

In [ ]:
# Distribution of Flights vs Type of Flight

g = sns.catplot(x="TIPOVUELO", data=dataset, kind="count")
g.set_axis_labels(
    "\nType of Flight (International or National)\n", "\nTotal Flights\n", size=16
)
g.fig.suptitle("\nNumber of Flights per Type of Flight", size=20, y=1.15)
g.figure.set_size_inches(15, 5)

In [ ]:
# calculate percent of National and International Flights

national_pct = dataset[dataset["TIPOVUELO"] == "N"]["TIPOVUELO"].count() / len(dataset)

print(f"{national_pct:.1%} of Flights are National")
print(f"{1-national_pct:.1%} of Flights are International")

In [ ]:
# Distribution of Flights per Operating Airline

g = sns.catplot(
    x="OPERA", data=dataset, kind="count", order=dataset["OPERA"].value_counts().index
)
g.set_axis_labels("\nOperating Airline\n", "\nTotal Flights\n", size=16)
g.fig.suptitle("\nNumber of Flights per Operating Airline", size=20, y=1.15)
g.figure.set_size_inches(15, 5)
# rotate xlabels
g.set_xticklabels(rotation=90)
for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(lambda x, p: f"{x:.0f}")

In [ ]:
# calculate percent for Airline 'Grupo LATAM' and 'Sky Airline
total_latam = dataset[dataset["OPERA"] == "Grupo LATAM"]["OPERA"].count()
total_sky = dataset[dataset["OPERA"] == "Sky Airline"]["OPERA"].count()

print(f"'Grupo LATAM' operates {total_latam/len(dataset):.1%} of the flights.")
print(f"'Sky Airline' operates {total_sky/len(dataset):.1%} of the flights.")
print(
    f"Together they account for {(total_sky+total_latam)/len(dataset):.1%} of flights."
)

In [ ]:
# Distribution of Flights per City of Destination

g = sns.catplot(
    x="SIGLADES",
    data=dataset,
    kind="count",
    order=dataset["SIGLADES"].value_counts().index,
)
g.set_axis_labels("\nCity of Destination\n", "\nTotal Flights\n", size=16)
g.fig.suptitle("\nNumber of Flights per City of Destination", size=20, y=1.15)
g.figure.set_size_inches(15, 5)
# rotate xlabels
g.set_xticklabels(rotation=90)
for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(lambda x, p: f"{x:.0f}")

### 1.1 Main Conclusions and Interpretation:

1. On average, Saturday (**Sabado**) is the day with the least flights of the whole week. The rest days are similar but slightly higher for Monday (**Lunes**), Thursday (**Jueves**) and Friday (**Viernes**).
2. There is a difference in number of flights per month of the year, there are months with low demand (April/4, May/5 and June/6) and high demand (July/7, October/10, November/11, December/12 and January/1).
3. Slightly more than half (54.2%) of the flights are of '**National**' Type, while the rest (45.8%) are '**International**'.
4. Most flights are operated by '**Grupo LATAM**' (60%), followed by '**Sky Airline**' (21%). Together they account for 80.9% of total flights.
5. As seen previously (*not plotted*), the only City of Origin is '**Santiago**' de Chile.
6. The 5 most frequent destinations overall are (in descending order): '*Buenos Aires*', '*Antofagasta*', '*Lima*', '*Calama*' and '*Puerto Montt*'.	
7. The 5 most frequent **International** destinations are (in descending order): '*Buenos Aires*', '*Lima*', '*Sao Paulo*', '*Ciudad de Panama*' and '*Mendoza*'.
8. The 5 most frequent **National** destinations are (in descending order): '*Antofagasta*', '*Calama*', '*Puerto Montt*', '*Concepción*' e '*Iquique*'.

## 2. Create additional synthetic features

In [ ]:
# create new synthetic features from the original data

synthetic_features = pd.DataFrame(
    [], columns=["temporada_alta", "dif_min", "atraso_15", "periodo_dia"]
)
synthetic_features

In [ ]:
# new feature 'dif_min': difference in minutes between 'Fecha-O' and 'Fecha-I'.

synthetic_features["dif_min"] = dataset["Fecha-O"] - dataset["Fecha-I"]
synthetic_features["dif_min"] = synthetic_features["dif_min"].apply(
    lambda x: float(x.total_seconds() / 60.0)
)
synthetic_features["dif_min"]

In [ ]:
# new feature 'atraso_15': 1 if dif_min > 15, 0 if not.

synthetic_features["atraso_15"] = synthetic_features["dif_min"].apply(
    lambda x: 1 if x > 15 else 0
)
synthetic_features["atraso_15"]

In [ ]:
# new feature 'temporada_alta': 1 if 'Fecha-I' is between 15-Dic and 3-Mar,
# or 15-Jul and 31-Jul, or 11-Sep and 30-Sep, 0 if not.

synthetic_features["temporada_alta"] = dataset["Fecha-I"].apply(
    lambda x: 1
    if (x > datetime(month=1, day=1, year=2017))
    and (x < datetime(month=3, day=3, year=2017))
    or (x > datetime(month=12, day=15, year=2017))
    and (x < datetime(month=3, day=3, year=2018))
    or (x > datetime(month=7, day=15, year=2017))
    and (x < datetime(month=7, day=31, year=2017))
    or (x > datetime(month=9, day=11, year=2017))
    and (x < datetime(month=9, day=30, year=2017))
    else 0
)

temporada_alta_pct = synthetic_features["temporada_alta"].sum() / len(dataset)
print(f"'temporada_alta' is true for {temporada_alta_pct:.1%} of the flights.")

In [ ]:
# new feature 'periodo_dia': 'mañana' (if time between 5:00-11:59),
# 'tarde' (between 12:00-18:59) y 'noche' ( between 19:00-4:59), using 'Fecha-I'.

synthetic_features["periodo_dia"] = dataset["Fecha-I"].apply(
    lambda x: "mañana"
    if (x.hour >= 5) and (x.hour < 12)
    else "tarde"
    if (x.hour >= 12) and (x.hour < 19)
    else "noche"
)
synthetic_features["periodo_dia"]

In [ ]:
# Export Synthetic Features to CSV file

synthetic_features.to_csv("output/synthetic_features.csv", index=False)
synthetic_features

## 3. Analyze Delay Features

In [ ]:
# join synthetic features into main dataframe

dataset = dataset.join(synthetic_features)
dataset.head()

In [ ]:
# Distribution of Average Delay per City of Destination

grouped_df = (
    dataset.groupby(by="SIGLADES", as_index=False)["dif_min"]
    .mean()
    .sort_values(by="dif_min", ascending=False)
)

g = sns.catplot(x="SIGLADES", y="dif_min", data=grouped_df, kind="bar")
g.set_axis_labels("\nCity of Destination\n", "\nAverage Delay in minutes\n", size=16)
g.fig.suptitle("\n Average Delay of Flights per City of Destination", size=20, y=1.15)
g.figure.set_size_inches(15, 5)
# rotate xlabels
g.set_xticklabels(rotation=90)
for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(lambda x, p: f"{x:.0f}")

In [ ]:
# Destinations with average delay > 15

grouped_df[grouped_df["dif_min"] > 15]

In [ ]:
# 10 destinations with less delay

grouped_df.tail(10)

In [ ]:
# Distribution of Average Delay per operating Airline

grouped_df = (
    dataset.groupby(by="OPERA", as_index=False)["dif_min"]
    .mean()
    .sort_values(by="dif_min", ascending=False)
)

g = sns.catplot(x="OPERA", y="dif_min", data=grouped_df, kind="bar")
g.set_axis_labels("\nOperating Airline\n", "\nAverage Delay in minutes\n", size=16)
g.fig.suptitle("\n Average Delay of Flights per Operating Airline", size=20, y=1.15)
g.figure.set_size_inches(15, 5)
# rotate xlabels
g.set_xticklabels(rotation=90)
for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(lambda x, p: f"{x:.0f}")

In [ ]:
# Airlines with average delay > 15

grouped_df[grouped_df["dif_min"] > 15]

In [ ]:
# 10 Airlines with less delay

grouped_df.tail(10)

In [ ]:
# Distribution of Average Delay per MONTH

grouped_df = (
    dataset.groupby(by="MES", as_index=False)["dif_min"]
    .mean()
    .sort_values(by="dif_min", ascending=False)
)

g = sns.catplot(x="MES", y="dif_min", data=grouped_df, kind="bar")
g.set_axis_labels("\nMONTH\n", "\nAverage Delay in minutes\n", size=16)
g.fig.suptitle("\n Average Delay of Flights per MONTH", size=20, y=1.15)
g.figure.set_size_inches(15, 5)

In [ ]:
# día de la semana
# Distribution of Average Delay per Day of the Week

grouped_df = dataset.groupby(by="DIANOM", as_index=False)["dif_min"].mean()

g = sns.catplot(
    x="DIANOM",
    y="dif_min",
    data=grouped_df,
    kind="bar",
    order=["Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado", "Domingo"],
)
g.set_axis_labels("\nDay of the Week\n", "\nAverage Delay in minutes\n", size=16)
g.fig.suptitle("\n Average Delay of Flights per Day of the Week", size=20, y=1.15)
g.figure.set_size_inches(15, 5)

In [ ]:
# Correlation between Average Delay vs Season

dataset[["temporada_alta", "dif_min"]].corr(method="spearman")

In [ ]:
# Distribution of Average Delay per Season

grouped_df = (
    dataset.groupby(by="temporada_alta", as_index=False)["dif_min"]
    .mean()
    .sort_values(by="dif_min", ascending=False)
)

g = sns.catplot(x="temporada_alta", y="dif_min", data=grouped_df, kind="bar")
g.set_axis_labels("\nHigh season\n", "\nAverage Delay in minutes\n", size=16)
g.fig.suptitle("\n Average Delay of Flights per Season (Low or High)", size=20, y=1.15)
g.figure.set_size_inches(15, 5)

In [ ]:
grouped_df["dif_min"][1] - grouped_df["dif_min"][0]

In [ ]:
# Correlation between Average Delay vs Flight Type

pd.DataFrame(
    {
        "TIPOVUELO": dataset["TIPOVUELO"].replace({"I": 1, "N": 0}),
        "dif_min": dataset["dif_min"],
    }
).corr(method="spearman")

In [ ]:
# Distribution of Average Delay per Flight Type

grouped_df = (
    dataset.groupby(by="TIPOVUELO", as_index=False)["dif_min"]
    .mean()
    .sort_values(by="dif_min", ascending=False)
)

g = sns.catplot(x="TIPOVUELO", y="dif_min", data=grouped_df, kind="bar")
g.set_axis_labels(
    "\nFlight Type (International or National)\n",
    "\nAverage Delay in minutes\n",
    size=16,
)
g.fig.suptitle("\n Average Delay per Flight Type", size=20, y=1.15)
g.figure.set_size_inches(15, 5)

In [ ]:
grouped_df["dif_min"][0] - grouped_df["dif_min"][1]

In [ ]:
# Distribution of Average Delay per period of day

grouped_df = dataset.groupby(by="periodo_dia", as_index=False)["dif_min"].mean()

g = sns.catplot(
    x="periodo_dia",
    y="dif_min",
    data=grouped_df,
    kind="bar",
    order=["mañana", "tarde", "noche"],
)
g.set_axis_labels("\nPeriod of day\n", "\nAverage Delay in minutes\n", size=16)
g.fig.suptitle("\n Average Delay of Flights per Period of day", size=20, y=1.15)
g.figure.set_size_inches(15, 5)

In [ ]:
grouped_df

### 3.1 Main Conclusions and Interpretation:

1. There are 11 destinations that are more than 15 minutes late on average, they are: 'Cochabamba', 'Quito', 'Puerto Stanley', 'Sydney', 'Ushuia', 'Melbourne', 'Rosario', 'Bariloche', 'Auckland N.Z.', 'Toronto' and 'Mendoza'.
2. The average delay for 'Cochabamba', 'Quito' and 'Puerto Stanley' is over 50 minutes.
3. The 10 less delayed destinatios are: 'Ciudad de Mexico', 'Pisco, Peru', 'Paris', 'Cataratas Iguacu', 'Ciudad de Panama', 'Atlanta', 'Washington', 'Dallas', 'Houston', 'Curitiba, Bra.'.
4. The average delay for 'Ciudad de Panama', 'Atlanta' and 'Washington' is less than 1 minute.
5. 'Dallas', 'Houston' and 'Curitiba, Bra.' depart more than 1 minute early on average.
6. The Airlines with more than 15 minutes of average delay are: 'Plus Ultra Lineas Aereas', 'Qantas Airways', 'Latin American Wings', 'Air Canada'.
7. The 10 less delayed Airlines are: 'Alitalia', 'Lacsa', 'Iberia', 'Air France', 'K.L.M.', 'American Airlines', 'Copa Air', 'Delta Air', 'United Airlines', 'Aeromexico'.
8. 'Air France', 'K.L.M.', 'American Airlines', 'Copa Air' and 'Delta Air' are delayed less than 2 minutes on average.
9. 'United Airlines' and 'Aeromexico' depart around 2 minutes early on average.
10. The most delayed Months are July/7, October/10 and December/12, with July having an average delay of more than 15 minutes.
11. The week days with highest average delay are Monday (Lunes) and Friday (Viernes). The least delays occur on Sunday (Domingo).
12. There is a very little difference of 0.82 minutes of additional delay on average between High and Low season
13. There is a positive but low 'spearman' correlation of 0.0275 between minutes of delay and high_season. This means that there is a little effect of season on delay but is not a strong feature on its own to predict delay. The Spearman correlation is used over the typical Pearson correlation because it assesses monotonic relationships (whether linear or not linear).
14. There is a little difference of 2.82 minutes of additional delay on average between the Type of Flight (International or National).
15. There is a positive but low 'spearman' correlation of 0.056 between minutes of delay and Type of Flight. This means that there is a little effect of season on delay but is not a strong feature on its own to predict delay.
16. The average delay on the morning flights is around 1.7 minutes less than on the afternoon and night.

### 3.2 What would be the most important features to predict delays? 

According to the previous analysis and observations, the best features would be those that showed a strong variation between the feature categories and the delay in minutes.

Features with big influence:
- City of Destination ('SIGLADES')
- Operating Airline ('OPERA)
- Month ('MES')
- Day of Week ('DIANOM')

Features with medium influence:
- Flight Type ('TIPOVUELO')
- Period of day ('periodo_dia')

Features with small influence:
- High season ('temporada_alta'): did not show so much strength, but using the exact dates could introduce a better signal for the model.

## 4. Create Machine Learning Model(s)
- The main goal is to estimate the probability of delay for a flight.
- Delay is assumed as a binary class with 1 meaning that the flight departed with > 15min delay.
- The Model must be a binary classifier and transform score into probability
- Optional: create another regression model to predict the exact delay in minutes. => minutes of delay could also be transformed into delay probability.

### 4.1 Include Additional Features

- did a change occur to the flight program ? (between programmed and operation data) => **Is this data known a priori?**
- use date as 'mes-dia' categorical feature, ignore year (careful not to over-fit)
- include list of holidays for chile, non-working days, school vacations, etc.
- include meteorological data for SCL Airport (Arturo Merino Benitez)
- Try various ML and Deep Neural Network models.
- Optimize hyperparameters to increase performance

In [ ]:
# Did the program change ?

dataset["Vlo-I"] != dataset["Vlo-O"]

### Specific Dates

In [ ]:
# Distribution of Average Delay per specific date (MONTH-DAY)

# create Month-Day feature
dataset["MES-DIA"] = dataset["MES"].astype(str) + "-" + dataset["DIA"].astype(str)

grouped_df = (
    dataset.groupby(by="MES-DIA", as_index=False)["dif_min"]
    .mean()
    .sort_values(by="dif_min", ascending=False)
)

g = sns.catplot(
    x="MES-DIA", y="dif_min", data=grouped_df[grouped_df["dif_min"] > 15], kind="bar"
)
g.set_axis_labels(
    "\nSpecific date (MONTH-DAY)\n", "\nAverage Delay in minutes\n", size=16
)
g.fig.suptitle(
    "\n Average Delay (> 15 minutes) per specific date (MONTH-DAY)", size=20, y=1.15
)
g.figure.set_size_inches(15, 5)
# rotate xlabels
g.set_xticklabels(rotation=90)
for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(lambda x, p: f"{x:.0f}")

Average Delay seems to be more associated with specific date combinations than simply using the High/Low season periods.

#### Metereological Data
According to a [report](https://www.transportation.gov/sites/dot.gov/files/docs/kulesa_Weather_Aviation.pdf) from the US Federal Aviation Administration (FAA), weather conditions cause ~70% of the delays in the US National Airspace System.

Because metereological conditions are very important variables that could cause delays, data for SCL Airport was downloaded from https://climatologia.meteochile.gob.cl/ and included in the `input/additional_data/` folder.

In [ ]:
# Relative Humidity per Hour
humidity = pd.concat(
    [
        pd.read_csv(
            "input/additional_data/330021_2017_Humedad_.csv.zip",
            compression="zip",
            sep=";|,",
        ),
        pd.read_csv(
            "input/additional_data/330021_2018_Humedad_.csv.zip",
            compression="zip",
            sep=";|,",
        ),
    ],
    axis=0,
    ignore_index=True,
)
humidity.rename(
    columns={"momento": "date", "HR_Valor": "relative_humidity"}, inplace=True
)
humidity["date"] = humidity["date"].apply(
    lambda x: datetime.strptime(x, "%d-%m-%Y %H:%M:%S")
)
humidity.set_index("date", inplace=True)
humidity["relative_humidity"] = humidity["relative_humidity"].apply(lambda x: x / 100)
humidity = humidity.interpolate(method="linear")
humidity.drop(columns=["CodigoNacional"], inplace=True)
humidity

In [ ]:
# Dew point (the temperature in which air becomes saturated with water vapor)

dew_point = pd.concat(
    [
        pd.read_csv(
            "input/additional_data/330021_2017_PuntoRocio_.csv.zip",
            compression="zip",
            sep=";|,",
        ),
        pd.read_csv(
            "input/additional_data/330021_2018_PuntoRocio_.csv.zip",
            compression="zip",
            sep=";|,",
        ),
    ],
    axis=0,
    ignore_index=True,
)
dew_point.rename(columns={"momento": "date", "Td_Valor": "dew_point"}, inplace=True)
dew_point["date"] = dew_point["date"].apply(
    lambda x: datetime.strptime(x, "%d-%m-%Y %H:%M:%S")
)
dew_point.set_index("date", inplace=True)
dew_point = dew_point.interpolate(method="time")
dew_point.drop(columns=["CodigoNacional"], inplace=True)
dew_point

In [ ]:
# Air Temperature per hour

temp = pd.concat(
    [
        pd.read_csv(
            "input/additional_data/330021_2017_Temperatura_.csv.zip",
            compression="zip",
            sep=";|,",
        ),
        pd.read_csv(
            "input/additional_data/330021_2018_Temperatura_.csv.zip",
            compression="zip",
            sep=";|,",
        ),
    ],
    axis=0,
    ignore_index=True,
)
temp.rename(columns={"momento": "date", "Ts_Valor": "temperature"}, inplace=True)
temp["date"] = temp["date"].apply(lambda x: datetime.strptime(x, "%d-%m-%Y %H:%M:%S"))
temp.set_index("date", inplace=True)
temp = temp.interpolate(method="time")
temp.drop(columns=["CodigoNacional"], inplace=True)
temp

In [ ]:
# Relative precipitation per Hour (6-hr interval interpolated to 1-hr interval)
precipitation = pd.concat(
    [
        pd.read_csv(
            "input/additional_data/330021_2017_Agua6Horas_.csv.zip",
            compression="zip",
            sep=";|,",
        ),
        pd.read_csv(
            "input/additional_data/330021_2018_Agua6Horas_.csv.zip",
            compression="zip",
            sep=";|,",
        ),
    ],
    axis=0,
    ignore_index=True,
)
precipitation.rename(columns={"momento": "date", "RRR6_Valor": "rain_mm"}, inplace=True)
precipitation["date"] = precipitation["date"].apply(
    lambda x: datetime.strptime(x, "%d-%m-%Y %H:%M:%S")
)
precipitation.set_index("date", inplace=True)
# reindex interval from 6 hours to 1 hour difference
precipitation = precipitation.reindex(
    pd.date_range("2017-01-01 00:00:00", "2018-12-31 23:00:00", freq="H")
)
# interpolate missing values then normalize to maintain the same total mm of rain
precipitation = precipitation.interpolate(method="time").apply(lambda x: x / 6)
precipitation.drop(columns=["CodigoNacional", "Traza_Valor"], inplace=True)
precipitation

Presence of Fog:

If the temperature of the air is around 2.5ºC of the dew point temperature, then the water vapor condensed will remain suspended in the air in the form of tiny droplets of water forming a fog. Also if Relative Humidty > 95%

In [ ]:
# Formation of Fog per Hour

fog = [
    1
    if abs(temp.loc[date, "temperature"] - dew_point.loc[date, "dew_point"]) <= 2.5
    or humidity.loc[date, "relative_humidity"] >= 0.95
    else 0
    for date in temp.index
]
fog = pd.DataFrame(fog, index=temp.index, columns=["fog"])
fog

Presence of Ice and Frost:

When temperature is below 2ºC as measured on air, the ground is close to or under 0ºC which forms ice (cold air descends).

If it's raining then there will be ice formed on the runway.

Also, if the air temperature is below the dew point, then the condensed water vapor will form dew that will turn into frost on the runway.

In [ ]:
# Formation of Frost per Hour
frost = [
    1
    if temp.loc[date, "temperature"] <= 2
    and (temp.loc[date, "temperature"] < dew_point.loc[date, "dew_point"])
    or precipitation.loc[date, "rain_mm"] > 0
    else 0
    for date in temp.index
]
frost = pd.DataFrame(frost, index=temp.index, columns=["frost"])
frost

In [ ]:
# Cloud Coverage per Day (resample to hour, interpolating data)
cloud_coverage = pd.concat(
    [
        pd.read_csv(
            "input/additional_data/330021_2017_CieloCubierto_.csv.zip",
            compression="zip",
            sep=",",
        ),
        pd.read_csv(
            "input/additional_data/330021_2018_CieloCubierto_.csv.zip",
            compression="zip",
            sep=",",
        ),
    ],
    axis=0,
    ignore_index=True,
)
cloud_coverage["Cloud Coverage %"] = cloud_coverage["Octavas de Cielo Cubierto"].apply(
    lambda x: x / 8
)
cloud_coverage.rename(columns={"Fecha": "date"}, inplace=True)
cloud_coverage["date"] = cloud_coverage["date"].apply(
    lambda x: datetime.strptime(x, "%Y-%m-%d")
)
cloud_coverage.set_index("date", inplace=True)
# change values to 1 hour, interpolating missing values
cloud_coverage = cloud_coverage.reindex(
    pd.date_range("2017-01-01 00:00:00", "2018-12-31 23:00:00", freq="H")
)
cloud_coverage = cloud_coverage.interpolate(method="time")
cloud_coverage.drop(columns="Octavas de Cielo Cubierto", inplace=True)
cloud_coverage

In [ ]:
# Wind per Hour
wind = pd.concat(
    [
        pd.read_csv(
            "input/additional_data/330021_2017_Viento_.csv.zip",
            compression="zip",
            sep=";|,",
        ),
        pd.read_csv(
            "input/additional_data/330021_2018_Viento_.csv.zip",
            compression="zip",
            sep=";|,",
        ),
    ],
    axis=0,
    ignore_index=True,
)
wind.rename(columns={"momento": "date", "ff_Valor": "wind_speed"}, inplace=True)
wind["date"] = wind["date"].apply(lambda x: datetime.strptime(x, "%d-%m-%Y %H:%M:%S"))
wind.set_index("date", inplace=True)
wind = wind.interpolate(method="time")
wind.drop(columns=["CodigoNacional", "dd_Valor", "VRB_Valor"], inplace=True)
wind

In [ ]:
# Atmospheric pressure
atm_pressure = pd.concat(
    [
        pd.read_csv(
            "input/additional_data/330021_2017_PresionQFE_.csv.zip",
            compression="zip",
            sep=";|,",
        ),
        pd.read_csv(
            "input/additional_data/330021_2018_PresionQFE_.csv.zip",
            compression="zip",
            sep=";|,",
        ),
    ],
    axis=0,
    ignore_index=True,
)
atm_pressure.rename(columns={"momento": "date", "QFE_Valor": "pressure"}, inplace=True)
atm_pressure["date"] = atm_pressure["date"].apply(
    lambda x: datetime.strptime(x, "%d-%m-%Y %H:%M:%S")
)
atm_pressure.set_index("date", inplace=True)
atm_pressure = atm_pressure.interpolate(method="time")
atm_pressure.drop(columns=["CodigoNacional"], inplace=True)
atm_pressure

In [ ]:
# Merge Meteorological data into one dataframe

weather = pd.concat(
    [cloud_coverage, atm_pressure, fog, frost, wind, temp, precipitation, humidity],
    axis=1,
)
# fill NaN's with previous value
weather.fillna(method="ffill", inplace=True)
weather

In [ ]:
# Are there still NaN's ?

weather.isna().sum().sum()

### Add Holiday data

In [ ]:
dataset[["Fecha-I", "AÑO", "MES", "DIA"]]

In [ ]:
# Load Holidays applicable to Chile

CL_holidays = holidays.Chile()
# list comprehension to mark holidays as 1 and non-holidays as 0
is_holiday = [1 if date in CL_holidays else 0 for date in dataset["Fecha-I"]]
# add column to dataset
dataset["is_holiday"] = is_holiday
dataset

In [ ]:
# Correlation between Average Delay vs Holidays

pd.DataFrame(
    {
        "is_holiday": dataset["is_holiday"],
        "dif_min": dataset["dif_min"],
    }
).corr(method="spearman")

Delay doesn't seem to be correlated to holidays (it even shows a weak negative corr)

### Add School Vacation data

In [ ]:
# new feature 'temporada_alta': 1 if 'Fecha-I' is between 15-Dic and 3-Mar,
# or 15-Jul and 31-Jul, or 11-Sep and 30-Sep, 0 if not.

dataset["is_school_vacation"] = dataset["Fecha-I"].apply(
    lambda x: 1
    if (x >= datetime(month=1, day=1, year=2017))
    and (x <= datetime(month=3, day=6, year=2017))
    or (x >= datetime(month=7, day=10, year=2017))
    and (x <= datetime(month=7, day=21, year=2017))
    or (x >= datetime(month=9, day=17, year=2017))
    and (x <= datetime(month=9, day=21, year=2017))
    or (x >= datetime(month=12, day=13, year=2017))
    else 0
)
dataset.head()

In [ ]:
# Correlation between Average Delay vs School Vacation

pd.DataFrame(
    {
        "is_school_vacation": dataset["is_school_vacation"],
        "dif_min": dataset["dif_min"],
    }
).corr(method="spearman")

There's only a weak positive correlation between School Vacations and Delays

In [ ]:
#  is the dataset balanced ?
g = sns.catplot(x="atraso_15", data=dataset, kind="count")
g.set_axis_labels(
    "\nDelay > 15 min\n",
    "\nTotal number of Flights\n",
    size=16,
)
g.fig.suptitle("\nDelay > 15 minutes per Total number of Flights", size=20, y=1.15)
g.figure.set_size_inches(15, 5)

In [ ]:
class_imbalance = (
    dataset[dataset["atraso_15"] == 0].count()[0]
    / dataset[dataset["atraso_15"] == 1].count()[0]
)
print(f"The class imbalance ratio is: {class_imbalance:.1f} to 1")

There is a big imbalance between the binary class to predict, which should be considered when evaluating the performance of the model (If not considered, a random binary model with 50% chance per class would predict the correct class 81.5% of the time with the current imbalance ratio).